## Artificial neural network 
### where can I get the data?
https://www.superdatascience.com/deep-learning/

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


import pandas as pd
import tensorflow as tf
# import theano
import keras 

import numpy as np
import matplotlib.pyplot as plt
from time import time

/home/shj16110/anaconda2/envs/3.6.1/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# import tensorflow as tf
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 378819940559892056
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 11323952333
locality {
  bus_id: 1
}
incarnation: 4289320405906153976
physical_device_desc: "device: 0, name: Tesla K40m, pci bus id: 0000:03:00.0"
]


In [4]:
# import tensorflow as tf
# with tf.device('/gpu:0'):
#     a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
#     b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
#     c = tf.matmul(a, b)

# with tf.Session() as sess:
#     print (sess.run(c))

In [5]:
!nvidia-smi

Tue Feb 20 17:07:39 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 375.26                 Driver Version: 375.26                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K40m          Off  | 0000:03:00.0     Off |                    0 |
| N/A   29C    P0    62W / 235W |  10873MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K40m          Off  | 0000:82:00.0     Off |                    0 |
| N/A   27C    P0    64W / 235W |      0MiB / 11439MiB |     99%      Default |
+-------------------------------+----------------------+----------------------+
        

In [6]:
def show(id, w=6):
    from IPython.display import Image
    return Image('./pics/class1/{}.jpg'.format(id), width=w*100)

In [7]:
!ls Artificial_Neural_Networks/

ann_homework_solution.py  classification_template.py
ann.py			  evaluating_improving_tuning.py
categorical_data.py	  Installations_Instructions.pdf


In [8]:
df = pd.read_csv('Churn_Modelling.csv')
df.shape

(10000, 14)

In [9]:
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [10]:
df.iloc[:,3:-1].head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58


In [11]:
df.iloc[:,3:13].head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58


In [12]:
df['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [13]:
df['Gender'].value_counts()

Male      5457
Female    4543
Name: Gender, dtype: int64

### data preprocessing

In [14]:
X = df.iloc[:,3:-1].values
print (X.shape)
print (type(X))
X[:3]

(10000, 10)
<class 'numpy.ndarray'>


array([[619, 'France', 'Female', 42, 2, 0.0, 1, 1, 1, 101348.88],
       [608, 'Spain', 'Female', 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [502, 'France', 'Female', 42, 8, 159660.8, 3, 1, 0, 113931.57]],
      dtype=object)

In [15]:
y = df.iloc[:,-1].values
print (y.shape)
print (type(y))
y

(10000,)
<class 'numpy.ndarray'>


array([1, 0, 1, ..., 1, 1, 0])

### Encoding categorical data

In [16]:
X[:,1:3]

array([['France', 'Female'],
       ['Spain', 'Female'],
       ['France', 'Female'],
       ...,
       ['France', 'Female'],
       ['Germany', 'Male'],
       ['France', 'Female']], dtype=object)

In [17]:
%%time

# Encoding the Independent Variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
print (X[:3])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
(X[:3])

[[619 0 'Female' 42 2 0.0 1 1 1 101348.88]
 [608 2 'Female' 41 1 83807.86 1 0 1 112542.58]
 [502 0 'Female' 42 8 159660.8 3 1 0 113931.57]]
CPU times: user 195 ms, sys: 47 ms, total: 242 ms
Wall time: 3.4 s


In [18]:
X

array([[619, 0, 0, ..., 1, 1, 101348.88],
       [608, 2, 0, ..., 0, 1, 112542.58],
       [502, 0, 0, ..., 1, 0, 113931.57],
       ...,
       [709, 0, 0, ..., 0, 1, 42085.58],
       [772, 1, 1, ..., 1, 0, 92888.52],
       [792, 0, 0, ..., 1, 0, 38190.78]], dtype=object)

In [19]:
%%time
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
print (X.shape)

(10000, 12)
CPU times: user 9 ms, sys: 2 ms, total: 11 ms
Wall time: 9.67 ms


In [20]:
onehotencoder.fit_transform(X)

<10000x13 sparse matrix of type '<class 'numpy.float64'>'
	with 91124 stored elements in COOrdinate format>

In [21]:
# %precision % .2f

In [22]:
onehotencoder.fit_transform(X).toarray()[0]

array([1.0000000e+00, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00,
       6.1900000e+02, 0.0000000e+00, 4.2000000e+01, 2.0000000e+00,
       0.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
       1.0134888e+05])

### How to pretty-printing a numpy.array without scientific notation and with given precision?

https://stackoverflow.com/questions/2891790/how-to-pretty-printing-a-numpy-array-without-scientific-notation-and-with-given

In [23]:
np.set_printoptions(precision=1, suppress=True)

In [24]:
onehotencoder.fit_transform(X).toarray()[0]

array([     1. ,      0. ,      1. ,      0. ,    619. ,      0. ,
           42. ,      2. ,      0. ,      1. ,      1. ,      1. ,
       101348.9])

### remove one dummy variable to avoid dummy variable trap

In [25]:
X[0]

array([     1. ,      0. ,      0. ,    619. ,      0. ,     42. ,
            2. ,      0. ,      1. ,      1. ,      1. , 101348.9])

In [26]:
print (X.shape)
X = X[:, 1:]
print (X.shape)
X[0]

(10000, 12)
(10000, 11)


array([     0. ,      0. ,    619. ,      0. ,     42. ,      2. ,
            0. ,      1. ,      1. ,      1. , 101348.9])

### train test split & feature scaling 

In [27]:
S = lambda *x: [i.shape for i in x]

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print (S(X_train,X_test, y_train, y_test))

[(8000, 11), (2000, 11), (8000,), (2000,)]


In [29]:
X_train[0]

array([     0. ,      1. ,    667. ,      0. ,     34. ,      5. ,
            0. ,      2. ,      1. ,      0. , 163830.6])

### nomalization

In [30]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# fit on training set
X_train = sc.fit_transform(X_train)
# only transform on test set
X_test = sc.transform(X_test)
print (S(X_train,X_test, y_train, y_test))
X_train[0]

[(8000, 11), (2000, 11), (8000,), (2000,)]


array([-0.6,  1.7,  0.2, -1.1, -0.5,  0. , -1.2,  0.8,  0.6, -1. ,  1.1])

## build ANN

In [31]:
# show(13, w=8)

In [32]:
from keras.models import Sequential
from keras.layers import Dense

In [33]:
classifier = Sequential()
# classifier?

In [34]:
# first hidden layer
# classifier.add(Dense(units = 6, input_shape=(11,),  kernel_initializer='uniform', activation='relu'))
classifier.add(Dense(units = 6, input_dim=11,  kernel_initializer='uniform', activation='relu'))

In [35]:
# second hidden layer
classifier.add(Dense(units = 6,  kernel_initializer='uniform', activation='relu'))

In [36]:
# ouput layer
classifier.add(Dense(units = 1,  kernel_initializer='uniform', activation='sigmoid'))

In [37]:
# show(19, w=8)

In [38]:
from keras.optimizers import adam, Adam

In [39]:
# compiling the ANN
classifier.compile(optimizer=adam(0.01), loss='binary_crossentropy', metrics= ['accuracy'])

In [40]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


### find process in Linux sort by memery usage

In [41]:
!ps aux --sort=-%mem | awk 'NR<=6{print $0}'

USER        PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root          1  0.0  0.0  23596   564 ?        Ss   Jan25   0:02 /sbin/init
root          2  0.0  0.0      0     0 ?        S    Jan25   0:00 [kthreadd]
root          3  0.0  0.0      0     0 ?        S    Jan25   2:04 [migration/0]
root          4  0.0  0.0      0     0 ?        S    Jan25   0:21 [ksoftirqd/0]
root          5  0.0  0.0      0     0 ?        S    Jan25   0:00 [stopper/0]


In [42]:
from keras.callbacks import EarlyStopping

In [43]:
# tf.all_variables()

In [44]:
# if 'session' in locals() and session is not None:
#     print('Close interactive session')
#     session.close()

In [45]:
# fitting ANN with training set
classifier.fit(X_train, y_train, batch_size=10, epochs=100, callbacks=[EarlyStopping(patience=10)])

Epoch 1/100
8000/8000 [==============================] - 3s 355us/step - loss: 0.4377 - acc: 0.8039
Epoch 2/100
 650/8000 [=>............................] - ETA: 1s - loss: 0.4053 - acc: 0.8277

/home/shj16110/anaconda2/envs/3.6.1/lib/python3.6/site-packages/keras/callbacks.py:497: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


8000/8000 [==============================] - 2s 238us/step - loss: 0.4194 - acc: 0.8269
Epoch 3/100
8000/8000 [==============================] - 2s 237us/step - loss: 0.4148 - acc: 0.8323
Epoch 4/100
8000/8000 [==============================] - 2s 236us/step - loss: 0.4151 - acc: 0.8323
Epoch 5/100
8000/8000 [==============================] - 2s 236us/step - loss: 0.4137 - acc: 0.8323
Epoch 6/100
8000/8000 [==============================] - 2s 235us/step - loss: 0.4130 - acc: 0.8305
Epoch 7/100
8000/8000 [==============================] - 2s 236us/step - loss: 0.4117 - acc: 0.8335
Epoch 8/100
8000/8000 [==============================] - 2s 234us/step - loss: 0.4123 - acc: 0.8324
Epoch 9/100
8000/8000 [==============================] - 2s 235us/step - loss: 0.4095 - acc: 0.8315
Epoch 10/100
8000/8000 [==============================] - 2s 235us/step - loss: 0.3846 - acc: 0.8434
Epoch 11/100
8000/8000 [==============================] - 2s 234us/step - loss: 0.3747 - acc: 0.8476
Epoch 12/1

8000/8000 [==============================] - 2s 235us/step - loss: 0.3463 - acc: 0.8592
Epoch 84/100
8000/8000 [==============================] - 2s 236us/step - loss: 0.3468 - acc: 0.8571
Epoch 85/100
8000/8000 [==============================] - 2s 234us/step - loss: 0.3459 - acc: 0.8566
Epoch 86/100
8000/8000 [==============================] - 2s 235us/step - loss: 0.3456 - acc: 0.8591
Epoch 87/100
8000/8000 [==============================] - 2s 234us/step - loss: 0.3477 - acc: 0.8559
Epoch 88/100
8000/8000 [==============================] - 2s 235us/step - loss: 0.3482 - acc: 0.8598
Epoch 89/100
8000/8000 [==============================] - 2s 235us/step - loss: 0.3492 - acc: 0.8564
Epoch 90/100
8000/8000 [==============================] - 2s 235us/step - loss: 0.3485 - acc: 0.8568
Epoch 91/100
8000/8000 [==============================] - 2s 235us/step - loss: 0.3474 - acc: 0.8590
Epoch 92/100
8000/8000 [==============================] - 2s 235us/step - loss: 0.3470 - acc: 0.8538
Epo

In [47]:
y_pred = classifier.predict(X_test)
print (type(y_pred))
y_pred.shape

<class 'numpy.ndarray'>


(2000, 1)

In [48]:
y_pred

array([[0.4],
       [0.3],
       [0.2],
       ...,
       [0.2],
       [0.2],
       [0.2]], dtype=float32)

In [49]:
y_pred[0][0]

0.38290864

In [50]:
y_pred = (y_pred> 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [51]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
type(cm)

numpy.ndarray

In [52]:
cm

array([[1483,  112],
       [ 182,  223]])

In [53]:
accuracy_score(y_test, y_pred)

0.853

In [54]:
(1512+211)/2000.0

0.8615

### Predicting a single new observation

In [58]:
# Predicting a single new observation
"""Predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000"""
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
print  (new_prediction)
new_prediction = (new_prediction > 0.5)
new_prediction

[[0.]]


array([[False]])

## cross validation